In [1]:
import json
import os
%load_ext autoreload
%autoreload



# i2b2 vs timebank
 ## EVENT
- i2b2: `<EVENT id="E0" start="1" end="10" text="ADMISSION" modality="FACTUAL" "polarity="POS" type="OCCURRENCE" />`
- tb-d: `<MAKEINSTANCE eventID="e1" eiid="ei252" tense="PAST" aspect="NONE" polarity="POS" pos="VERB"/>`



|          | Event | i2b2 | tb-d | Useful? |
|----------|-------|------|------|---------|
| modarity |       | v    | v    |         |
| polarity |       | v    | v    |         |
| type *(med)|       | v    |      | maybe - ner |
| tense    |       |      | v    |         |
| aspect   |       |      | v    |         |
| pos      |       |      | v


    | maybe   |

## TLlink
- i2b2: `<TLINK id="TL18" fromID="E9" fromText="outpatient chemotherapy" toID="E0" toText="ADMISSION" type="BEFORE" />`
- tb-d:  `<TLINK lid="l7" relType="AFTER" eventInstanceID="ei258" relatedToEventInstance="ei257"/>`

|             | Tlink | i2b2 | tb-d | Useful?    |
|-------------|-------|------|------|------------|
| t-l e-e (tt) link type|       | v (only this has tt)    | v    | maybe-ner**since both have it |
| rel type     |       | v (note type is rel type, not link type)    |      | maybe      |


<mark> # NOTE </mark>
- candicates for aligning with **type** in PEER which is entity type
    - try **link type** first since both data has it
    - then, try **event's (med) type** for i2b2, as it's more analogous to PEER's entity type
    - then pos 
        - available in tb
        - think about how to get it for i2b2
        - is pos used at all in PEER??
            - also consider refering to the other 2 datasets in PEER
                - did they use pos? (scierc seems to not use it at all)
                - how did they got pos

# TODO 1: prepare i2b2
# I will do i2b2 first, since it has both event type, and rel type for trying experiments
    - try use medical event first - direct alignment with entity type: you know outpatient is a surgery process and so it happens after admission?
    - then use tl link type, coz aligns with the intuition that you need to know outpatient is event, it happens after admission event?
    - composite of the two types

## Next TODO for i2b2
    - have the first two sets of experiements running
    - in the meantime trying to read and understand the PSL paper and think about temporal inference
        

In [2]:

# TODO clinical nlp I actually also need to understand the script to compute accuracy and f1 score

# TODO 2: prepare tbd

In [43]:

# TB-dense_data relatively small so I could consider running this first
import spacy
import json
import os

spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")
    
temp_tb_file = "/data/chengc7/TEDataProcessing/tbd_output"
js_lists = [json.loads(line) for line in open(temp_tb_file)]
types = ["train", "dev", "test"]

one_js = js_lists[0]
print(one_js.keys())
final_out = {'train':[], 'dev':[], 'test':[]}
ndoc_dict, nrel_dict = {}, {}
for i, json_doc in enumerate(js_lists):
    #for k, v in js_lists[t]:
        ndoc, nrel = 0, 0
        doc_id, raw_text, ents, rels, type = json_doc['id'], json_doc['raw_text'], json_doc["entities"], json_doc['relations'], json_doc['split'] 
        doc = nlp(raw_text)
        def get_sentence_entities(doc, ents):
            """
            return 
                - sentences_tokens : [[t_1, t_2,...], ..]
                - entities_dict: dict['eid'] = [[span.s, span.e, type, eType, sent_id, sent_start]]
                    - eg. {'E0': [0, 0, 'OCCURRENCE', 'EVENT', 0, 0],...}
                - sent_start_map: dict[sent_id] = sent_start (doc_level)
            """
            def get_event_doc_span(st, ed, doc):    
                """_summary_

                Args:
                    st (int): char level start id 
                    ed (int): char level end id 
                    doc (spacy.doc): 

                Returns:
                    doc level span
                    
                Note: a lot of weird cases for event=S1 (discharge) that needs shifting
                """
                span = doc.char_span(st, ed)
                doc_span = [span.start, span.end]
                assert doc[doc_span[0]:doc_span[1]].text== doc.text[st:ed]
                return doc_span
            # set doc_span for each entity
            for e_k, e_v in ents.items():
                doc_span = get_event_doc_span(e_v['span'][0], e_v['span'][1], doc)
                doc_span[-1] = doc_span[-1] - 1 # NOTE: adjusted doc_span end to match PEER
                e_v['doc_span'] = doc_span   
            entities = sorted(ents.items(), key=lambda x: x[1]['doc_span'][0])

            sentence_tokens = []
            #sent_span = []
            sent_start_map = {}
            entities_dict = {}
            e_idx = 0
            
            for sent_id, sent in enumerate(doc.sents):
                sentence_tokens.append([token.text for token in sent])
                #print( [sent.start_char, sent.end_char], [sent.start, sent.end], sent)
                #sent_span.append([sent.start, sent.end])
                sent_start_map[sent_id] = sent.start
                while e_idx < len(entities) and entities[e_idx][1]['doc_span'][0] < sent.end: 
                    e_k, e_v = entities[e_idx]
                    entities_dict[e_k] = e_v['doc_span']+[e_v['type'], e_v['eType']] + [sent_id] + [sent.start] # DEBUG: this sent.start are used to create sent_adjusted 
                    e_idx += 1

            assert len(entities) == e_idx == len(entities_dict)
            return sentence_tokens,  entities_dict, sent_start_map
        sentences, entities, sent_start_map = get_sentence_entities(doc, ents)

        print("test")
        #formulated_doc = {'doc_key':doc_id, 'sentences':tokens, 'relations':formulated_rels ,'ner':ner, }
        #final_out[type].append(formulated_doc)
        #print(f"**{k}**: v:{v}\n")
        
        
        
# for t in types[0:]: # debug one type
#     ndoc, nrel = 0, 0
#     with open(os.path.join(psl_tbd_dir, f"{t}.json"), "rb") as f:
#         data = json.load(f)    
#         for doc_id, doc_content in data.items():
#             ndoc += 1
#             for sent_range, sent_content in doc_content.items():
#                 text, rels = sent_content['text'], sent_content['relation']
#                 nrel += len(rels)
#                 #print("")
                
#     print(f"{t}: ndoc:{ndoc}, nrel:{nrel}")



dict_keys(['id', 'raw_text', 'entities', 'relations', 'split'])


AttributeError: 'NoneType' object has no attribute 'start'

In [36]:
text

"\n The Pentagon said today it  will re-examine the question are the remains inside the Tomb of the Unknown from the Vietnam War, in fact, known?\n\n CBS News first reported last night that the tomb may contain the remains of Air Force pilot Michael Blassie.\n\n There was a suspicion the body was Blassie because his uniform and ID card  were found near the body in Vietnam. But subsequently, they  were lost. Blassie's mother now wants the remains in the tomb tested for DNA.\n"

# JointEventTempRel:https://github.com/PlusLabNLP/JointEventTempRel/tree/master/data/tbd

In [1]:
import pickle
from dataclasses import dataclass

types = ["train", "dev", "test"]
from torch.utils import data

@dataclass
class Event():
    id: str
    type: str
    text: str
    tense: str
    polarity: str
    span: (int, int)
for t in types: # debug one type
    ndoc, nrel = 0, 0
    data_dir = "/data/chengc7/JointEventTempRel/data/tbd/"
    #data = EventDataset(data_dir, data_split=t)
    with open(os.path.join(data_dir, f"{t}.pickle"), "rb") as f:
        data = pickle.load(f)    
    for r_id, r_val in data.items():
        rel_type, doc_dictionary = r_val.rel_type
    #     for doc_id, doc_content in data.items():
    #         ndoc += 1
    #         for sent_range, sent_content in doc_content.items():
    #             text, rels = sent_content['text'], sent_content['relation']
    #             nrel += len(rels)
    #             #print("")
                
    print(f"{t}: ndoc:{ndoc}, nrel:{nrel}")
data.keys()



/vf/users/chengc7/conda/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'os' is not defined

In [34]:
r_val

{'rel_type': 'INCLUDES',
 'rev': False,
 'doc_dictionary': OrderedDict([('[1:7)', ('MOSCOW', 'NNP')),
              ('[8:9)', ('(', '-LRB-')),
              ('[9:11)', ('AP', 'NNP')),
              ('[11:12)', (')', '-RRB-')),
              ('[13:14)', ('_', 'VBP')),
              ('[15:25)', ('Presidents', 'NNS')),
              ('[26:32)', ('Leonid', 'NNP')),
              ('[33:39)', ('Kuchma', 'NNP')),
              ('[40:42)', ('of', 'IN')),
              ('[43:50)', ('Ukraine', 'NNP')),
              ('[51:54)', ('and', 'CC')),
              ('[55:60)', ('Boris', 'NNP')),
              ('[61:68)', ('Yeltsin', 'NNP')),
              ('[69:71)', ('of', 'IN')),
              ('[72:78)', ('Russia', 'NNP')),
              ('[79:85)', ('signed', 'VBD')),
              ('[86:88)', ('an', 'DT')),
              ('[89:97)', ('economic', 'JJ')),
              ('[98:109)', ('cooperation', 'NN')),
              ('[110:114)', ('plan', 'NN')),
              ('[115:121)', ('Friday', 'NNP')),
   

In [4]:
one_js['entities']


{'e1': {'id': 'e1',
  'type': 'EVENT',
  'text': 'Taking',
  'tense': 'PRESPART',
  'aspect': None,
  'polarity': 'POS',
  'span': [18, 23]},
 'e2': {'id': 'e2',
  'type': 'EVENT',
  'text': 'said',
  'tense': 'PAST',
  'aspect': None,
  'polarity': 'POS',
  'span': [92, 95]},
 'e3': {'id': 'e3',
  'type': 'EVENT',
  'text': 'regretted',
  'tense': 'PAST',
  'aspect': None,
  'polarity': 'POS',
  'span': [113, 121]},
 'e4': {'id': 'e4',
  'type': 'EVENT',
  'text': 'calling',
  'tense': 'PRESPART',
  'aspect': None,
  'polarity': 'POS',
  'span': [123, 129]},
 'e7': {'id': 'e7',
  'type': 'EVENT',
  'text': 'traitor',
  'tense': 'NONE',
  'aspect': None,
  'polarity': 'POS',
  'span': [152, 158]},
 'e8': {'id': 'e8',
  'type': 'EVENT',
  'text': 'praying',
  'tense': 'PRESPART',
  'aspect': None,
  'polarity': 'POS',
  'span': [164, 170]},
 'e9': {'id': 'e9',
  'type': 'EVENT',
  'text': 'death',
  'tense': 'NONE',
  'aspect': None,
  'polarity': 'POS',
  'span': [180, 184]},
 'e10': {

In [15]:
import pickle, os, json
psl_tbd_dir = "psl_tbd"
types = ["train", "dev", "test"]
id_pair_test = []
save_test_nrel = 0
for t in types: # debug one type
    ndoc, nrel = 0, 0
    with open(os.path.join(psl_tbd_dir, f"{t}.json"), "rb") as f:
        data = json.load(f)    
        for doc_id, doc_content in data.items():
            ndoc += 1
            for sent_range, sent_content in doc_content.items():
                text, rels = sent_content['text'], sent_content['relation']
                nrel += len(rels)
                #print("")]
                if t=='test':
                    for r in rels:
                        #_, _, 'e121', _, _, 'e125', 'BEFORE'
                        _, _, e1_id, _, _, e2_id, rel = r 
                        id_pair_test.append([doc_id, e1_id, e2_id, rel])
    if t=='test':
        save_test_nrel=nrel
    print(f"{t}: ndoc:{ndoc}, nrel:{nrel}")
data.keys()

assert len(id_pair_test) == nrel

tbd_data_dir = "tbd_data/relations/"
save_psl_ids_file = os.path.join(tbd_data_dir, "psl_event_ids.pickle")
with open(save_psl_ids_file, "wb") as f:
    pickle.dump(id_pair_test, f)


train: ndoc:22, nrel:3429
dev: ndoc:5, nrel:540
test: ndoc:9, nrel:1242


In [22]:
# t = "test"
# with open(os.path.join(tbd_data_dir, f"{t}.json"), "w") as f:
#     data = json.load(f)  
# # for doc in tbd_data_dir:
    

UnsupportedOperation: not readable

In [18]:

data['APW19980308.0201'].keys()


dict_keys(['[1:9)', '[205:208)', '[346:348)', '[664:671)', '[775:777)', '[906:912)'])

In [7]:
data['APW19980308.0201']['[1:9)'].keys()

dict_keys(['text', 'relation'])

# TODO 3
# NOTE: remember after preparing data, need to alter the SUBJ/OBJ part, are there too many pairs?
- need to swap the label from before to after when swap S/O
- also for time pairs, cannot do no relation, there is probably a relation by infering ...
- then think about enrich subj/obj with pos